# Determining whether predicted cleavage peptides (DeepPeptide) that don't match peptides in databases have other traits that support their veracity.

Some cleavage peptides (~49%) predicted by the DeepPeptide tool had matches to peptides in the Peptipedia database.
Peptipedia is a metadatabase comprised of peptide sequences from 76 databases.
This notebook investigates whether peptides that did not have matches contain other traits that support these peptides being real.

**Signal peptides**: Most (but not all) annotated cleavage peptides are cleaved from precursor proteins that contain an N-terminal signal peptide [https://doi.org/10.1038/s41467-022-34031-z].
Signal peptides target a protein to the secretory pathway and allow cleaved peptides to reach their final destination [https://doi.org/10.1096/fasebj.8.9.8005390].
Many cleavage peptides function as hormones or other signaling molecules, making export from the cell a key step in their biogenesis [https://doi.org/10.1096/fasebj.8.9.8005390]. 

**Propeptides**: Some precursor proteins include propeptides, which are segments that may help in the correct folding of the protein, inhibit premature activity before reaching the target site, or aid in the proper localization of the enzyme [https://doi.org/10.1002/prot.26702].
The propeptides are cleaved off to activate the protein or peptide once it has reached its destination.

We could investigate other signals (disulfide bonds, glycosylation sites, sorting signals), but these two were the easiest with data/tools we already had access to, so we started with these two.

## Prep notebook

In [115]:
library(tidyverse, warn.conflicts = F)

## Read in and process data

In [117]:
human_predictions <- read_tsv("../../peptigate/results/predictions/peptide_predictions.tsv.gz", show_col_types = F) %>%
  mutate(peptide_class = ifelse(is.na(peptide_class), "sORF", peptide_class)) %>%
  # Remove propeptide predictions, as propeptides don't have biological activity once cleaved.
  filter(peptide_class != "Propeptide") %>%
  # This notebook focuses on DeepPeptide cleavage peptides, so filter to these
  filter(prediction_tool == "deeppeptide")

human_annotations <- read_tsv("../../peptigate/results/predictions/peptide_annotations.tsv.gz", show_col_types = F) %>%
  mutate(length = nchar(sequence)) %>%
  mutate(peptipedia_blast_result = ifelse(!is.na(peptipedia_blast_bitscore), "blast hit", "no blast hit")) 

human_results <- left_join(human_predictions, human_annotations, by = "peptide_id")

## Filter to distinct peptide sequences

While there is no overlap in sequences predicted by different tools (code not shown), duplicate sequences arise from a tool (ex. DeepPeptide) predicting the same peptide sequence from different transcripts.
These transcripts are usually isoforms of the same gene that contain the same sequence that gives rise to the peptide.

Peptigate predicted 2,949 distinct peptide amino acid sequences.

In [118]:
# This code block filters to distinct sequences while keeping the most metadata possible.
# This requires removing metadata columns that might not be the same even if sequences are the same.
# We remove columns that we expect to vary like "peptide_id", "start", and "end".
human_results_distinct <- human_results %>%
  select(peptide_type, prediction_tool, sequence, length, 
         peptipedia_blast_pident, peptipedia_blast_evalue, 
         peptipedia_blast_bitscore,  peptipedia_blast_result) %>%
  distinct()

In [119]:
# Confirm the number of rows in the dataframe match the number of distinct sequences
length(unique(human_results_distinct$sequence))
nrow(human_results_distinct)

[1] 263

[1] 263

## How many deeppeptide-predicted peptides had a BLAST hit?

In [120]:
human_results_distinct %>%
  group_by(prediction_tool, peptipedia_blast_result) %>%
  tally() 

prediction_tool,peptipedia_blast_result,n
<chr>,<chr>,<int>
deeppeptide,blast hit,130
deeppeptide,no blast hit,133


## Join to signal peptide information

As documented in the README, we ran DeepSig on the precursor/parent proteins from which the peptides were cleaved from.
We started with the peptigate intermediate file that reports the sequences of the parent proteins: `cleavage/deeppeptide/deeppeptide_peptide_parents.faa`.
We then ran DeepSig and formatted the output using the peptigate script:
```
deepsig -f deeppeptide_peptide_parents.faa -o deeppeptide_peptide_parents.tmp -k euk
python peptigate/scripts/add_header_to_deepsig_tsv.py deeppeptide_peptide_parents.tmp deeppeptide_peptide_parents_deepsig.tsv
```

In [96]:
deepsig_deeppeptide <- read_tsv("tmp_deepsig_human/deeppeptide_peptide_parents_deepsig.tsv", show_col_types = F) %>%
  filter(deepsig_feature == "Signal peptide") %>%
  select(precursor_id = peptide_id, deepsig_parent = deepsig_feature) %>%
  mutate(prediction_tool = "deeppeptide")

deepsig_nlpprecursor <- read_tsv("tmp_deepsig_human/nlpprecursor_peptide_parents_deepsig.tsv", show_col_types = F) %>%
  filter(deepsig_feature == "Signal peptide") %>%
  select(parent_id = peptide_id, deepsig_parent = deepsig_feature) %>%
  mutate(prediction_tool = "nlpprecursor")

deepsig <- bind_rows(deepsig_deeppeptide, deepsig_nlpprecursor)
    
human_results2 <- human_results %>%
  # generate precursor protein sequence id from peptide id
  mutate(parent_id = gsub("_start.*", "", peptide_id)) %>%
  left_join(deepsig)

# re-derive distinct sequences so things aren't counted twice
human_results2_distinct <-  human_results2 %>%
  select(peptide_type, prediction_tool, sequence, AB, ACE, ACP, 
         AF, AMAP, AMP, AOX, APP, AV, BBP, DPPIV, MRSA, Neuro, QS, TOX, TTCA, 
         aliphatic_index, boman_index, charge, hydrophobicity, instability_index, 
         isoelectric_point, molecular_weight, pd1_residue_volume, 
         pd2_hydrophilicity, z1_lipophilicity, z2_steric_bulk_or_polarizability,
         z3_polarity_or_charge, z4_electronegativity_etc, z5_electronegativity_etc, 
         peptipedia_blast_pident, peptipedia_blast_evalue, 
         peptipedia_blast_bitscore,  peptipedia_blast_result, length, deepsig_combined, deepsig_parent) %>%
  mutate(deepsig_peptide = ifelse(grepl(pattern = "Signal", x = deepsig_combined), "Signal peptide", "Chain")) %>%
  distinct()

Joining with `by = join_by(prediction_tool, precursor_id)`


In [97]:
# note that there is duplication
nrow(human_results2_distinct)

[1] 2973

In [98]:
# get an overview of results
human_results2_distinct %>%
  group_by(prediction_tool, peptipedia_blast_result, deepsig_peptide, deepsig_parent) %>%
  tally()

prediction_tool,peptipedia_blast_result,deepsig_peptide,deepsig_parent,n
<chr>,<chr>,<chr>,<chr>,<int>
deeppeptide,blast hit,Chain,Signal peptide,67
deeppeptide,blast hit,Chain,NA,83
deeppeptide,no blast hit,Chain,Signal peptide,28
deeppeptide,no blast hit,Chain,NA,104
deeppeptide,no blast hit,Signal peptide,NA,4
nlpprecursor,blast hit,Chain,Signal peptide,8
nlpprecursor,blast hit,Chain,NA,80
nlpprecursor,no blast hit,Chain,Signal peptide,5
nlpprecursor,no blast hit,Chain,NA,336


In [100]:
# pull out specifically the number of distinct cleavage peptides whose precursor protein had a signal peptide
deeppeptide_no_blast_hit_but_signal_peptide <- human_results2_distinct %>%
  filter(prediction_tool == "deeppeptide") %>%
  filter(peptipedia_blast_result == "no blast hit") %>%
  filter(deepsig_parent == "Signal peptide")

length(unique(deeppeptide_no_blast_hit_but_signal_peptide$sequence))


[1] 28

## determine if cleavage peptides also have propeptides predicted from parent sequences

In [104]:
propeptide_predictions <- read_tsv("../../peptigate/results/predictions/peptide_predictions.tsv.gz", show_col_types = F) %>%
  filter(peptide_class == "Propeptide") %>%
  mutate(parent_id = gsub("_start.*", "", peptide_id))

In [114]:
human_results2 %>%
  mutate(parent_id = gsub("_start.*", "", peptide_id)) %>%
  mutate(has_propeptide = ifelse(parent_id %in% propeptide_predictions$parent_id, "propeptide", "no propeptide")) %>%
  filter(prediction_tool == "deeppeptide") %>%
  select(peptide_type, prediction_tool, sequence,
         aliphatic_index, boman_index, charge, hydrophobicity, instability_index, 
         isoelectric_point, molecular_weight, pd1_residue_volume, 
         pd2_hydrophilicity, z1_lipophilicity, z2_steric_bulk_or_polarizability,
         z3_polarity_or_charge, z4_electronegativity_etc, z5_electronegativity_etc, 
         peptipedia_blast_pident, peptipedia_blast_evalue, 
         peptipedia_blast_bitscore,  peptipedia_blast_result, length, deepsig_combined, has_propeptide, deepsig_parent) %>%
  distinct() %>%
  group_by(prediction_tool, peptipedia_blast_result, deepsig_parent, has_propeptide) %>%
  tally()

prediction_tool,peptipedia_blast_result,deepsig_parent,has_propeptide,n
<chr>,<chr>,<chr>,<chr>,<int>
deeppeptide,blast hit,Signal peptide,no propeptide,42
deeppeptide,blast hit,Signal peptide,propeptide,27
deeppeptide,blast hit,NA,no propeptide,60
deeppeptide,blast hit,NA,propeptide,24
deeppeptide,no blast hit,Signal peptide,no propeptide,20
deeppeptide,no blast hit,Signal peptide,propeptide,8
deeppeptide,no blast hit,NA,no propeptide,100
deeppeptide,no blast hit,NA,propeptide,8


In [99]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-apple-darwin13.4.0 (64-bit)
Running under: macOS Big Sur ... 10.16

Matrix products: default
BLAS/LAPACK: /Users/taylorreiter/miniconda3/envs/pepeval/lib/libopenblasp-r0.3.26.dylib;  LAPACK version 3.12.0

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] Rtsne_0.17          umap_0.2.10.0       arcadiathemeR_0.1.0
 [4] ggExtra_0.10.0      lubridate_1.9.3     forcats_1.0.0      
 [7] stringr_1.5.1       dplyr_1.1.4         purrr_1.0.2        
[10] readr_2.1.5         tidyr_1.3.1         tibble_3.2.1       
[13] ggplot2_3.5.1       tidyverse_2.0.0    

loaded via a namespace (and not attached):
 [1] gtable_0.3.4      ggrepel_0.9.5     lattice_0.22-6    tzdb_0.4.0       
 [5] vctrs_0.6.5       tools_4.3.3       generics_0.1.3    parallel_4.